## OpenStreetMap Data Case Study with SQL

Author: Qingyu Li

Date: Sep 26 2017

## 1. Map Area

Philadelphia, United States

- The OpenStreetMap Project: https://www.openstreetmap.org/relation/188022

- Data Extract: https://mapzen.com/data/metro-extracts/metro/philadelphia_pennsylvania/

I have lived in Philadelphia for a while and I would like to learn more about this fun city. 

In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

## 2. Data Check and Cleaning

After downloading the dataset for Philadelphia, we select a sample of the datafile and look for areas that need additional cleaning:
- Inconsistency in street name: 'St' for 'Street', 'Pkwy' for 'Parkway'. We need to update these names to make the street name consistent across all data points
- Misspelling of street names: 'Lane' is spelled as 'Line', 'Street' spelled as 'Sstreets',etc.
- City and state name are included in the street name. For example, "Baltimore Pike, Springfield, PA".

### Auditing Street Names

We use the audit function to return a list of the street names and collect a list of abbriviations that need to be updated.

In [2]:
OSMFILE = "philadelphia_pennsylvania.osm"
street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)


expected = ["Street", "Avenue", "Boulevard", "Drive", "Court", "Place", "Square", "Lane", "Road", 
            "Trail", "Parkway", "Commons"]

def audit_street_type(street_types, street_name):
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    return (elem.attrib['k'] == "addr:street")

def audit(osmfile):
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])
    osm_file.close()
    return street_types

pprint.pprint(dict(audit(OSMFILE))) 

{'1': set(['Bloomfield Dr, Unit 1',
           'Route 1',
           'S Newtown Street Rd #1',
           'Sansom St #1',
           'Walnut St #1']),
 '111': set(['South Clinton Avenue Ste. 111']),
 '118': set(['Upland Ave #118']),
 '168': set(['Marlton Pike East Ste. 168']),
 '17': set(['Lancaster Avenue #17']),
 '19047': set(['200 Manor Ave. Langhorne, PA 19047',
               '2245 E. Lincoln Hwy, Langhorne, PA 19047',
               '2275 E Lincoln Hwy, Langhorne, PA 19047',
               '2300  East Lincoln Highway, Pennsylvania 19047']),
 '19067': set(['East Trenton Avenue Morrisville, PA 19067']),
 '2': set(['Buck Rd #2']),
 '205': set(['Office Center Dr #205']),
 '206': set(['US 206', 'US 70 & US 206']),
 '3': set(['Main St #3']),
 '302': set(['Route 73 North, Suite 302']),
 '312': set(['312']),
 '315': set(['Heritage Center Dr #315']),
 '33': set(['33', 'Route 33']),
 '37th': set(['N 37th']),
 '38': set(['New Jersey 38', 'Route 38', 'State Route 38']),
 '39th': set(['N 39th

From the list, we can see that 'St.' and 'St' are needed to updated to 'Street', 'Ln' to 'Lane', 'Rd' to 'Road', and etc. There are also misspelling in street names:"Line" should be written as "Lane". In addition, some address contains the state name and they need to be removed. The following program make the update and print out the results:

In [34]:
mapping = { "St": "Street",
            "St.": "Street",
           "Sstreet":"Street",
            "Ave": "Avenue",
           "Ave.": "Avenue",
             "avenue": "Avenue",
            "ave": "Avenue",
            "Blvd": "Boulevard",
            "Blvd.": "Boulevard",
           "blvd": "Boulevard",
            "Bouevard": "Boulevard",
            "Boulavard": "Boulevard",
            "Boulvard": "Boulevard",
            "Rd.": "Road",
            "Rd": "Road",
           "RD": "Road",
           "rd": "Road",
           "road": "Road",
            "st": "Street",
           "Street.": "Street", 
           "street": "Street",
           "Sts":"Street",
           "Pkwy": "Parkway", 
           "Ct": "Court", 
           "Cir": "Circle",
           "Dr": "Drive", 
           "Dr.": "Drive", 
           "Hwy": "Highway",
           "Hwy.": "Highway",
           "Ln": "Lane",
           "Line":"Lane",
           "L":"Lane",
            "Ln.": "Lane",
           "PIke":"Pike",
            "Pl": "Place",
           "PLACE": "Place",
           "place":"Place",
            "Plz": "Plaza",
            "Rd": "Road",
            "Rd.": "Road",
            "St": "Street",
            "St.": "Street",
            "st": "Street",
           "ST": "Street",
            "street": "Street",
            "square": "Square",
            "parkway": "Parkway",
            "N.": "North",
            "N": "North",
            "E.": "East",
            "E": "East",
            "S.": "South",
            "S": "South",
            "W.": "West",
            "W": "West",
           "way":"Way"
            }

def update_name(name, mapping):

    # YOUR CODE HERE
    m=street_type_re.search(name)
    if m not in expected:
        if m.group() in mapping.keys():
            name = re.sub(m.group(), mapping[m.group()], name)
    name = re.split(",|\#|\-",name)[0]
    return name
    

st_types = audit(OSMFILE) 

In [35]:
for st_type, ways in st_types.iteritems():
    for name in ways:
        better_name = update_name(name, mapping)
        print name, "=>", better_name

White Horse => White Horse
Redstone Ridge => Redstone Ridge
1 Brookline BlvdHavertown, PA 19083(610) 446-1234 => 1 Brookline BlvdHavertown
Pennel Road; Pennsylvania Route 452 => Pennel Road; Pennsylvania Route 452
Market Street; Pennsylvania Route 452 => Market Street; Pennsylvania Route 452
Hillcrest Heights => Hillcrest Heights
52 Centerton Rd => 52 Centerton Road
Clements Bridge Rd => Clements Bridge Road
58 Centerton Rd => 58 Centerton Road
E County Line Rd => E County Line Road
Darby Rd => Darby Road
Stokes Rd => Stokes Road
Conestoga Rd => Conestoga Road
Church Rd => Church Road
York Rd => York Road
Dilworthtown Rd => Dilworthtown Road
62 Centerton Rd => 62 Centerton Road
South Easton Rd => South Easton Road
Durham Rd => Durham Road
Lincoln Mill Rd => Lincoln Mill Road
Evesham Rd => Evesham Road
Anderson Rd => Anderson Road
Barren Hill Rd => Barren Hill Road
Valley Rd => Valley Road
W Bristol Rd => W Bristol Road
Bristol Rd => Bristol Road
24 Centerton Rd => 24 Centerton Road
Edi

St John St => Street John Street
North Front St => North Front Street
S 20th St => S 20th Street
507 E Tulpehocken St => 507 E Tulpehocken Street
Walnut St => Walnut Street
Cooper St => Cooper Street
S 28th St => S 28th Street
S 47th St => S 47th Street
S 41st St => S 41st Street
North Broad St => North Broad Street
Swede St => Swede Street
Bush St => Bush Street
N Main St => N Main Street
N Broad St => N Broad Street
S 24th St => S 24th Street
E Hampton St => E Hampton Street
W Ritner St => W Ritner Street
McKean St => McKean Street
State Route 38 => State Route 38
Route 38 => Route 38
New Jersey 38 => New Jersey 38
Marlton Pike East Ste. 168 => Marlton Pike East Ste. 168
Pennsylvania Turnpike => Pennsylvania Turnpike
West king => West king
Lancaster Avenue #17 => Lancaster Avenue 
Ashley Preston => Ashley Preston
N Preston => N Preston
North 13th and Brown Streets => North 13th and Brown Streets
Country Walk => Country Walk
Hamilton Walk => Hamilton Walk
Liacouras Walk => Liacouras W

Here we can see that all the issue with street names are updated.

## 3. Prepare SQL Database

After data cleaning, we save the nodes, tags, and ways into csv file and import into sqlite to create a database containing these tables.

## 4. Data Overview

The sizes of files used in the project are as follows:


|File Name                      |        Size     |
|-------------------------------|-----------------|
|Philadelphia_pennsylvania.osm  |  737.1 MB       |
|philadelphia.db                |  531.4 MB       |
|nodes.csv                      |  274.8 MB       |
|nodes_tags.csv                 |  20.2 MB        |
|ways_nodes.csv                 |  94.6 MB        |
|ways.csv                       |  20.6 MB        |
|ways_tags.csv                  |  54.1 MB        |

Other statistics explored using sqlite3:

In [2]:
import csv, sqlite3
con = sqlite3.connect("philadelphia.db")
cur = con.cursor()

Number of nodes in the data:

In [7]:
result=cur.execute('SELECT COUNT(*) FROM nodes')
result.fetchone()[0]

3289070

Number of ways in the data:

In [9]:
result = cur.execute('SELECT COUNT(*) FROM ways')
result.fetchone()[0]

343066

Number of unique users:

In [22]:
result = cur.execute('SELECT COUNT(DISTINCT(e.uid)) \
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e')
result.fetchall()[0]

(2340,)

## 5. Data Exploration

Number of common ammenities in the data:

In [24]:
cur.execute('SELECT value, COUNT(*) as num \
            FROM nodes_tags \
            WHERE key="amenity" \
            GROUP BY value \
            ORDER BY num DESC \
            LIMIT 10').fetchall()


[(u'school', 1613),
 (u'restaurant', 1024),
 (u'place_of_worship', 704),
 (u'fire_station', 439),
 (u'fast_food', 396),
 (u'bench', 378),
 (u'social_facility', 279),
 (u'parking', 245),
 (u'fuel', 244),
 (u'cafe', 238)]

We can see that the top 1 ammenity in Philadelphia is school, which has 1613. And restaurant follows with 1024 locations.

Number of popular cuisines in the city:

In [26]:
cur.execute('SELECT nodes_tags.value, COUNT(*) as num \
            FROM nodes_tags \
                JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value="restaurant") i \
                ON nodes_tags.id=i.id \
            WHERE nodes_tags.key="cuisine" \
            GROUP BY nodes_tags.value \
            ORDER BY num DESC\
            LIMIT 10').fetchall()

[(u'pizza', 119),
 (u'italian', 56),
 (u'chinese', 49),
 (u'mexican', 41),
 (u'american', 39),
 (u'sandwich', 28),
 (u'burger', 18),
 (u'asian', 17),
 (u'indian', 12),
 (u'japanese', 12)]

We can see that the cuisine people like the most in Philadelphia is pizza, next is italian and the third is chinese food.

## 6. Conclusion

The OpenStreetMap data of Philadelphia is of good quality. We found some typos in street spelling and some address contain city and state names as well as inconsistency in abbreviations in the street type. We spent a significant amount of time checking different fields and cleaning the dataset. However, there are still a lot of extra work needed to improve the data quality of this extract. 

### Suggestions for standardizing street names and control typos:
- We can work on some preset rules for data imput. Preset the common names selection, such as "Street", "Place", "Lane", "Pike", "Parkway", etc. 
- We can develop a function to automatically clean the data periodically. This may be tedious at first, but with all data imput well controlled and database well maintained, the database will be easier to use and users do not need to spend a lot of time working on data cleaning.

# 7. Files

- Philadelphia_pennsylvania.osm: data downloaded from the OpenStreetMap project
- audit.py: audit and update street names
- data.py: parse and shape the osm data and create CSV files from OSM dataset
- dbwrite.py: read in the CSV files and create sql database 
- mapparser.py: find unique tags in the data
- query.py: data exploration using sqlite

   
   